# Import Libraries

In [1]:
import os
import numpy as np
import bs4
import time
import requests
import lxml
import urllib
import urllib.request

from pprint import pprint
from bs4 import BeautifulSoup
from tqdm import tqdm
from getpass import getpass
from urllib.parse import urljoin, urlparse

# Parse all websites of the Manga

In [2]:
# one pirce: https://ww9.readonepiece.com/manga/one-piece-digital-colored-comics/
# my hero academia: https://ww7.readmha.com/manga/boku-no-hero-academia-colored/

## colored

In [3]:
html_text = requests.get('https://ww7.readmha.com/manga/boku-no-hero-academia-colored/', 'r').text
soup = BeautifulSoup(html_text, 'lxml')

cl_chapters = soup.find_all('div', class_ = 'bg-bg-secondary p-3 rounded mb-3 shadow')
cl_chapter_site = []
for chapter in cl_chapters:
    site = chapter.find('a', class_ = 'uppercase text-xs bg-text-success text-white font-bold py-1 px-2 rounded').attrs['href']
    cl_chapter_site.append(site)

# cl_chapter_site.sort()
pprint(len(cl_chapter_site))
pprint(cl_chapter_site[0])
pprint(cl_chapter_site[-1])

64
'https://ww7.readmha.com/chapter/boku-no-hero-academia-colored-chapter-347'
'https://ww7.readmha.com/chapter/boku-no-hero-academia-colored-chapter-125'


## grayscale

In [4]:
html_text = requests.get('https://ww7.readmha.com/manga/boku-no-hero-academia/', 'r').text
soup = BeautifulSoup(html_text, 'lxml')

bw_chapters = soup.find_all('div', class_ = 'bg-bg-secondary p-3 rounded mb-3 shadow')
bw_chapter_site = []
for chapter in bw_chapters:
    site = chapter.find('a', class_ = 'uppercase text-xs bg-text-success text-white font-bold py-1 px-2 rounded').attrs['href']
    bw_chapter_site.append(site)

# bw_chapter_site.sort()
pprint(len(bw_chapter_site))
pprint(bw_chapter_site[0])
pprint(bw_chapter_site[-1])

372
'https://ww7.readmha.com/chapter/boku-no-hero-academia-chapter-354'
'https://ww7.readmha.com/chapter/boku-no-hero-academia-chapter-0'


In [5]:
cl_chapters = []
bw_chapters = []

for i in tqdm(range(len(cl_chapter_site)), 'colored, chapter 900 and above'):
    cl_chapters.append(cl_chapter_site[i])
        
for j in tqdm(range(len(bw_chapter_site) - 300), 'grayscale, chapter 900 and above'):
    bw_chapters.append(bw_chapter_site[j])

grayscale, chapter 900 and above: 100%|██████████| 72/72 [00:00<00:00, 345131.30it/s]


In [6]:
pprint(len(cl_chapters))
pprint(len(bw_chapters))

pprint(cl_chapters[-3:None])
pprint(bw_chapters[-3:None])

64
72
['https://ww7.readmha.com/chapter/boku-no-hero-academia-colored-chapter-127',
 'https://ww7.readmha.com/chapter/boku-no-hero-academia-colored-chapter-126',
 'https://ww7.readmha.com/chapter/boku-no-hero-academia-colored-chapter-125']
['https://ww7.readmha.com/chapter/boku-no-hero-academia-chapter-286',
 'https://ww7.readmha.com/chapter/boku-no-hero-academia-chapter-285',
 'https://ww7.readmha.com/chapter/boku-no-hero-academia-chapter-284']


# Parsing all image urls from each chapter

## colored

In [7]:
cl = []
for i in tqdm(range(len(cl_chapters)), "extracting images"):
# for i in tqdm(range(1), "extracting images"):
    cl_imgs = []
    
    html_text = requests.get(cl_chapters[i], 'r').text
    soup = BeautifulSoup(html_text, 'lxml')

    all_imgs = soup.find_all('div', class_ = 'text-center')
    for img in all_imgs:
        site = img.find("img", class_ = 'mb-3 mx-auto js-page')
        if site is not None:
            cl_imgs.append(site['src'])
            
    cl = cl + cl_imgs[1:None]
            
#     pprint(cl_imgs)
#     pprint(cl)
#     raise ValueError("done")

extracting images: 100%|██████████| 64/64 [01:02<00:00,  1.03it/s]


In [8]:
print(len(cl))
print(cl[0])

1050
https://i.imgur.com/x4BaC0S.jpeg


## grayscale

In [9]:
bw = []
for i in tqdm(range(len(bw_chapters)), "extracting images"):
# for i in tqdm(range(1), "extracting images"):
    bw_imgs = []
    
    html_text = requests.get(bw_chapters[i], 'r').text
    soup = BeautifulSoup(html_text, 'lxml')

    all_imgs = soup.find_all('div', class_ = 'text-center')
    for img in all_imgs:
        site = img.find("img", class_ = 'mb-3 mx-auto js-page')
        if site is not None:
            bw_imgs.append(site['src'])    
            
    bw = bw + bw_imgs[1:None]

extracting images: 100%|██████████| 72/72 [01:13<00:00,  1.02s/it]


In [10]:
print(len(bw))
print(bw[0])

1094
https://cdn.readmha.com/file/CDN-M-A-N/bnha_353_01.png


# Download the images

## colored

In [11]:
pathname = './colored'
# if path doesn't exist, make that path dir
if not os.path.isdir(pathname):
    os.makedirs(pathname)
    
for i in tqdm(range(len(cl)), "downloading images"):
    url = cl[i].replace('\r', '')  
#     url = 'https://cdn.readonepiece.com/file/mangap/3258/10900000/1.jpeg'
    filename = os.path.join(pathname, str(i) + str('.png'))

    response = requests.get(url)
    if response.ok:    
        with open(filename, 'wb') as f:
            f.write(requests.get(url).content)
    else:
        print(response)

downloading images: 100%|██████████| 1050/1050 [35:55<00:00,  2.05s/it]


## grayscale

In [12]:
pathname = './grayscale'
# if path doesn't exist, make that path dir
if not os.path.isdir(pathname):
    os.makedirs(pathname)
    
for i in tqdm(range(len(bw)), "downloading images"):
    url = bw[i].replace('\r', '')
    filename = os.path.join(pathname, str(i) + str('.png'))

    response = requests.get(url)
    if response.ok:    
        with open(filename, 'wb') as f:
            f.write(requests.get(url).content)
    else:
        print(response)

downloading images: 100%|██████████| 1094/1094 [07:57<00:00,  2.29it/s]
